In [234]:
import pennylane as qml
from copy import deepcopy
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [235]:
# data = np.loadtxt("iris.txt")
# X = data[:, 0:2]
# print("First X sample (original)  :", X[0])

# # pad the vectors to size 2^2 with constant values
# padding = 0.3 * np.ones((len(X), 1))
# X_pad = np.c_[np.c_[X, padding], np.zeros((len(X), 1))]
# print("First X sample (padded)    :", X_pad[0])

# # normalize each input
# normalization = np.sqrt(np.sum(X_pad ** 2, -1))
# X_norm = (X_pad.T / normalization).T
# print("First X sample (normalized):", X_norm[0])

# # angles for state preparation are new features
# features = np.array([get_angles(x) for x in X_norm])
# print("First features sample      :", features[0])

# Y = data[:, -1]

import pandas as pd

df = pd.read_csv("./DISE.csv")
df = df[['school_id', 'st_ratio', 'sComp_ratio', 'intervention']]
df['st_ratio'] = df['st_ratio'] / max(df['st_ratio'])
df['sComp_ratio'] = df['sComp_ratio'] / max(df['sComp_ratio'])
df1 = df[df['intervention'] == 1].sample(10)
df0 = df[df['intervention'] == 0].sample(10)
# df['intervention'] = df['intervention']
df = df1.append(df0)
df['intervention'] = df['intervention'].astype(float)
df['intervention'] = df['intervention'].replace(0, -1)

X = np.array([[x1,x2] for x1, x2 in zip(df['st_ratio'], df['sComp_ratio'])])
Y = np.array(df['intervention'].to_list())

# pad the vectors to size 2^2 with constant values
padding = 0.3 * np.ones((len(X), 1))
X_pad = np.c_[np.c_[X, padding], np.zeros((len(X), 1))]
print("First X sample (padded)    :", X_pad[0])

# normalize each input
normalization = np.sqrt(np.sum(X_pad ** 2, -1))
X_norm = (X_pad.T / normalization).T
print("First X sample (normalized):", X_norm[0])

# angles for state preparation are new features
features = np.array([get_angles(x) for x in X_norm])
print("First features sample      :", features[0])

First X sample (padded)    : [0.19158497 0.03452422 0.3        0.        ]
First X sample (normalized): [0.53571243 0.09653708 0.83886398 0.        ]
First features sample      : [ 1.9903858  -0.          0.         -0.17828973  0.17828973]


In [236]:
dev = qml.device("default.qubit", wires=2)

def get_angles(x):

    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(
        np.sqrt(x[2] ** 2 + x[3] ** 2)
        / np.sqrt(x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2)
    )

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])

@qml.qnode(dev)
def circuit(weights, x):
    qml.RY(x[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(x[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(x[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(x[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(x[4], wires=1)
    qml.PauliX(wires=0)

    for W in weights:
        qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
        qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
        qml.CNOT(wires=[0, 1])

    return qml.sample(qml.PauliZ(0))




In [238]:
def cost(y, y_pred):
    return np.sum((y_pred - y)**2)


def model(features, var):
    return circuit(var[0], features).mean()# + var_init[1]


def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if p < 0:
            p = -1
        else:
            p = 1
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss


def calc_gradient(x, y, var_init, shift):

    grads = []

    for layer1 in range(len(var_init[0])):
        for layer2 in range(len(var_init[0][layer1])):
            for angle in range(len(var_init[0][layer1][layer2])):

                new_weights_rs, new_weights_ls = deepcopy(var_init[0]), deepcopy(var_init[0])

                new_weights_rs[layer1][layer2][angle] = new_weights_rs[layer1][layer2][angle] + shift
                new_weights_ls[layer1][layer2][angle] = new_weights_ls[layer1][layer2][angle] - shift

                cost_rs = cost(y, model(x, (new_weights_rs, 0)))
                cost_ls = cost(y, model(x, (new_weights_ls, 0)))
                
                gradient = (cost_rs - cost_ls) / (2*(np.pi/2))
                grads.append(gradient)

    grads = np.reshape(np.array(grads), (6, 2, 3))
    return grads


def calc_mean_grad(x, y, var_init, shift):
    sum_grads = np.zeros((6,2,3))
    for i in range(len(x)):
        cur_grad = calc_gradient(x[i], y[i], var_init, np.pi/2)
        sum_grads += cur_grad
    return sum_grads / len(x)

In [243]:
num_qubits = 2
num_layers = 6
shift = np.pi / 2
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)

In [249]:
lr = 0.1

for epoch in range(20):

    print("EPOCH: ", epoch)

    x = features[7:12]
    y = Y[7:12]

    y_preds = [model(i, var_init) for i in x]

    # print("Prediction: ", y_preds)

    loss = cost(y, y_preds)

    print("    Loss: ", loss)

    acc = accuracy(y, y_preds) 

    print("    Accuracy: ", acc)

    gradient = calc_mean_grad(x, y, var_init, np.pi/2)

    var_init = ((var_init[0] - (lr * gradient)), 0)

    print("\n")


EPOCH:  0
    Loss:  9.161392
    Accuracy:  0.4


EPOCH:  1
    Loss:  9.01502
    Accuracy:  0.4


EPOCH:  2
    Loss:  8.553012
    Accuracy:  0.4


EPOCH:  3
    Loss:  8.224908000000001
    Accuracy:  0.4


EPOCH:  4
    Loss:  7.9198200000000005
    Accuracy:  0.4


EPOCH:  5
    Loss:  7.66556
    Accuracy:  0.4


EPOCH:  6
    Loss:  7.513276
    Accuracy:  0.4


EPOCH:  7
    Loss:  7.02164
    Accuracy:  0.4


EPOCH:  8
    Loss:  6.334876
    Accuracy:  0.4


EPOCH:  9
    Loss:  6.035528
    Accuracy:  0.2


EPOCH:  10
    Loss:  5.680284
    Accuracy:  0.4


EPOCH:  11
    Loss:  5.5999360000000005
    Accuracy:  0.2


EPOCH:  12
    Loss:  5.688472
    Accuracy:  0.4


EPOCH:  13
    Loss:  5.476188
    Accuracy:  0.6


EPOCH:  14
    Loss:  5.3969
    Accuracy:  0.6


EPOCH:  15
    Loss:  5.686928
    Accuracy:  0.6


EPOCH:  16
    Loss:  5.84924
    Accuracy:  0.6


EPOCH:  17
    Loss:  6.070639999999999
    Accuracy:  0.6


EPOCH:  18
    Loss:  6.058996
    Accurac